## Import necessary python modules

In [1]:
import os
import napari
import tifffile
import numpy as np
import matplotlib.pyplot as plt

import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack

from copy import deepcopy
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ, reorderZstack, getSpotAndClusters_multi

In [2]:
from order_spot import * 

## Specify Input and Output folders, identifiers

In [3]:
homeFolder = '/media/raphael/data/works/20231220_C3_14_2h_ofON/new_correction/'
nameKey = 'C3.14_ON_01_w1SPI-488_s'

imsQ = '4'
cellNumber = '18'
nucleiStackForm = nameKey+imsQ+"_cell_"

pathToTimeSequenceFullView = homeFolder+nameKey+imsQ+'/*.tif'
cell_Crop_Folder = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/'
pathToTimeSequenceCell = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/*.tif'


sequenceCell = imr(pathToTimeSequenceCell)
sequenceCell = sequenceCell
mipSequenceCell = np.max(sequenceCell, axis=1)
MaxTimePoint = sequenceCell.shape[0]

## Specify voxel and object size

In [4]:
voxelRadius = (700, 110, 110) # in nanometers
objectRadius = (700, 105, 105) # in nanometers

In [5]:
alpha = 0.5

beta = 2
gamma = 15

## Choose detection threshold

In [6]:
selectedThreshold = int(input("Enter your selected threshold: "))

Enter your selected threshold: 70


In [7]:
minFrame = int(input("Enter your max Frame: "))
minFrame = minFrame 

Enter your max Frame: 1


In [8]:
maxFrame = int(input("Enter your max Frame: "))
maxFrame = maxFrame + 1 

Enter your max Frame: 120


### Compute list of threshold 

## Get a list of spots detected using the threshold specified in previous step

Voire graphe du notebook 4 bleaching correction

Toutes les frames ne suivent pas la meme allure avec des images bcp moins intense que l'autre d'une frame a l'autre besoin de ponderer le threshold en fonction d'une moyenne generale et d'une liste de moyenne en fonction des frames 

if mean_frame < moy_gen :
    abaisser le threshold
elid mean_frame > moy_gen :
    augmenter le threshold
    
else : 
    conserver threshold 
    
Ou possibilite d'appliquer ce principe a une difference de pourcentage d'une frame a l'autre 

In [9]:
images=[]    
spots_list=[]

spot_radius_px = detection.get_object_radius_pixel(
    voxel_size_nm=voxelRadius, 
    object_radius_nm=objectRadius, 
    ndim=3)

for t in range(minFrame,maxFrame,1):

    path = os.path.join(cell_Crop_Folder, nucleiStackForm+str(cellNumber)+'_t'+str(f"{t:03}")+".tif")
    rna = stack.read_image(path)
    images.append(rna)

n=len(images)
print("Total number of images : "+str(n))

selectedThreshold_loop = selectedThreshold

for rna in images:
    try : 
        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)
    
        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)
    
        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        
        selectedThreshold_loop = (selectedThreshold_loop) #*((100+var_list[i])/100) 
        #print(selectedThreshold_loop)
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold_loop))
        spots_list.append(spots_)
    except IndexError as e: 
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold))
        spots_list.append(spots_)
        print('Done')
        break 

KeyboardInterrupt: 

## Build reference spot

In [ ]:
reference_spot = buildReferenceSpotFromImages(images, spots_list, alpha=alpha, gamma=5, voxelSize=voxelRadius, objectSize=objectRadius)
%matplotlib inline
plt.figure(num=3,figsize=(3,3))
plt.imshow(np.max(reference_spot, axis=0))
refSpot = deepcopy(reference_spot)

## Perform spot and cluster detection for all frames

comparer detection avec beta = 1 gamma = 10 et beta = 2 gamma = 20 

In [ ]:
spotsFrame, clustersFrames, ThresholdFrames = getSpotAndClusters(images, 
                                                                 reference_spot, 
                                                                 cellnumber=cellNumber, 
                                                                 startTime=minFrame-1,
                                                                 stopTime=maxFrame-1,
                                                                 thresholdManual=selectedThreshold, 
                                                                 beta=beta, #2
                                                                 gamma=gamma, #15 
                                                                 numberOfSpots=2,
                                                                 radiusCluster=400, 
                                                                 voxelSize=voxelRadius, 
                                                                 objectSize=objectRadius,
                                                                 extensionMov='.tif')

In [ ]:
a = sort_clustersFrames(clustersFrames,mipSequenceCell,400,perc=1)

## Correct clusters position if needed 

a[X] = move_sorted_clustersFrames(a[X], [z,y,x,n,i], new_position=int)

In [ ]:
# a[2] = move_sorted_clustersFrames(a[2], np.array([ 7, 81, 63,  3,  2]), 0)

## Merge clusters 

In [ ]:
b = merge_clusters_frame(a,diff=3)

In [ ]:
c = create_index2(b)    

In [ ]:
c

In [ ]:
a[61]

In [ ]:
def get_all_clusters(images,threshold,size):
    '''
    This function return a list of array corresponding to all the pixel corresponding to the most intense regions of a cell 
    '''
    res=[]
    i = 0
    for frames in images : 
        tmp = np.where(frames>threshold)
        coord = []
        for px in range(0,len(tmp[0])):
            z = tmp[0][px]
            y = tmp[1][px]
            x = tmp[2][px]
            if len(coord) < size : 
                coord.append([z,y,x,i])
        res.append(coord)
        i = i+1 
    return res 

In [ ]:
ttest = get_all_clusters(images,400,50)

## Save detection results to be used later

In [ ]:
saveSpotsNPZ(np.array(spotsFrame, dtype=object), 
             np.array(clustersFrames, dtype=object), 
             np.array(ThresholdFrames, dtype=object), 
             cellNumber, 
             cell_Crop_Folder, 
             reference_spot,
             threshold = selectedThreshold
             ) 


## Visualise detection results

In [ ]:
def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
    cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)


pts_coordinates = spotsFrame
cluster_coordinate = c

viewer = napari.Viewer()
image_layer = viewer.add_image(
        mipSequenceCell, colormap='green' #maxImageCell
        )
if image_layer.data.ndim == 4:
    bigfishSpots = spotsFrame
elif image_layer.data.ndim == 3:
    bigfishSpots = getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2))
    
bigfish_Spots = viewer.add_points(
        getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2)-1),
        face_color='#00000000',
        size=4,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='white',
        name = 'bigFish Detected Spots'
        )

bigfish_clusters = viewer.add_points(
        getDetectedClustersForFrame(cluster_coordinate,int(np.shape(mipSequenceCell)[0]/2)-1),
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='red',
        symbol='diamond',
        name = 'bigFish Clusters'
        )


viewer.dims.events.current_step.connect(
        lambda event: set_pts_features(bigfish_Spots, bigfish_clusters, pts_coordinates, cluster_coordinate, event.value)
        )



# Save clusters frame in csv 

In [ ]:
b = np.concatenate(c)
savepath = homeFolder+nameKey+imsQ + '/' + nameKey + imsQ + '_TS' +'_cell_' + cellNumber + '_fusion_'+  str(alpha) +'_' + str(beta) + '_' + str(gamma) + '_thres_' + str(selectedThreshold) + '.csv'

In [ ]:
entete = ['z','y','x','n_molecule','index']

In [ ]:
import csv 

with open(savepath, 'w', newline='') as csv_file :
    writer = csv.writer(csv_file)
    writer.writerow(entete)
    writer.writerows(b)

In [ ]:
import session_info
session_info.show()